# 4-1. 관심영역
 - 이미지 전체 영역에 연산을 적용할 때보다 관심있는 부분만 잘라내서 연산을 적용할 때의 이점:
   * 연산수행시간 ↓
   * 데이터 양 ↓
   * 알고리즘 단순해짐.
   * 영역이 작을 때 이미지의 좌표를 구하기 쉬움.

#### 4-1-1. 관심영역 지정
 - Region Of Interest(ROI): 전체 이미지에서 연산과 분석의 대상이 되는 영역.
 - OpenCV C++과 달리 관심영역 지정을 위한 별도의 API 없음 ==> Numpy Slicing을 이용하면 아주 편리! 

 - 전체 이미지: 변수 img
 - 관심있는 영역 좌표: (x, y)
 - 영역의 폭, 높이: w, h
    
       roi = img[y:y+h, x:x+w]
    
   => img의 y행부터 y+h행까지, x열에서 x+w열까지 슬라이싱한 것! 

 - ★ Numpy를 이용해 관심영역을 지정할 때 주의할 두 가지 사항.
   * 1. Numpy배열은 row, column 순으로 접근하므로 반드시 height → width 순으로 지정해야 한다.
   * 2. 새로운 리스트 객체를 반환하는 파이썬 리스트의 slicing과 달리, Numpy 배열의 slicing은 원본의 참조를 반환한다.
      
    ==> 원본과 무관한 새로운 작업을 하려면 copy()로 이미지 배열의 복제본을 만들어야 한다.

In [ ]:
#practice. 관심영역 지정

import cv2
import numpy as np

img = cv2.imread('../img/sunset.jpg')
x=320; y=150; w=50; h=50           #roi 좌표
roi = img[y:y+h, x:x+w]            #roi 지정

print(roi.shape)                   #roi shape, (50, 50, 3)
cv2.rectangle(roi, (0, 0), (h-1, w-1), (0, 255, 0))    #roi에 사각형 그리기
cv2.imshow("img", img)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
#practice. 관심영역 복제 및 새 창 띄우기
import cv2
import numpy as np

img = cv2.imread('../img/sunset.jpg')
x = 320; y = 150; w = 50; h = 50
roi = img[y:y+h, x:x+w]
img2 = roi.copy()
img[y:y+h, x+w:x+w+w] = roi  #새로운 좌표에 roi 추가, 태양 2개만들기.
cv2.rectangle(img, (x, y), (x+w+w, y+h), (0, 255, 0)) #2개의 태양 영역에 사각형 표시.

cv2.imshow("img", img)    #원본 이미지 출력
cv2.imshow("roi", img2)      #roi만 따로 출력.

cv2.waitKey(0)
cv2.destroyAllWindows()

#### 4-1-2. 마우스로 관심영역 지정
 - 마우스로 원하는 영역을 직접 지정하고 좌표를 알아냄.
 - 마우스 이벤트 활용.

In [10]:
#practice. 마우스로 관심영역 지정
# 마우스 이벤트 처리를 적용, 마우스로 관심영역을 지정하고 잘라낸 부분만 새 창에 표시하고 파일로 저장.

import cv2
import numpy as np

isDragging = False                  #마우스 드래그 상태 저장
x0, y0, w, h = -1, -1, -1, -1       #영역 선택 좌표 저장
blue, red = (255, 0, 0), (0, 0, 255) #색상값

def onMouse(event, x, y, flags, param):  #마우스 이벤트 핸들 함수 -- ①
    global isDragging, x0, y0, img       #전역변수 참조
    if event == cv2.EVENT_LBUTTONDOWN:   #왼쪽마우스 버튼 다운, 드래그 시작 -- ②
        isDragging = True                   #드래그가 시작되었다는 것을 기억하기 위해 isDragging 변경
        x0 = x                              #처음 마우스를 클릭하는 지점을 x0, y0 전역변수에 저장
        y0 = y
    elif event == cv2.EVENT_MOUSEMOVE:   #마우스 움직임 -- ③
        if isDragging:                   #드래그 진행중일 때
            img_draw = img.copy()        #사각형 그리기 전 이미지 복사(같은 원본에 사각형이 누적되는 것 방지)
                                         #아무것도 그려지지 않은 원본 이미지를 매번 새롭게 복사해서 거기에 사각형을 표시.
            cv2.rectangle(img_draw, (x0, y0), (x, y), blue, 2)  #드래그 진행 영역 표시(파란색)
            cv2.imshow('img', img_draw)  #사각형으로 표시된 그림 화면 출력
    elif event == cv2.EVENT_LBUTTONUP:   #왼쪽 마우스 버튼 업 -- ④
        if isDragging:                   #드래그 중지
            isDragging = False           #드래그 상태를 원래대로 돌려놓는다.
            w = x - x0                   #드래그 영역 폭 계산(현재 x좌표 - 최초 x좌표)
            h = y - y0                   #드래그 영역 높이 계산(현재 y좌표 - 최초 y좌표)
            print("x:%d, y:%d, w:%d, h:%d" % (x0, y0, w, h) )  #<-- 드래그한 관심영역.
        if w > 0 and h > 0:              #폭, 높이가 양수이면 드래그 방향이 옳음 ---⑤
            img_draw = img.copy()        #선택 영역에 사각형 그림을 표시할 이미지 복제
            #선택 영역에 빨간색 사각형 표시
            cv2.rectangle(img_draw, (x0, y0), (x, y), red, 2)
            cv2.imshow('img', img_draw)  #빨간색 사각형이 그려진 이미지 화면 출력
            roi = img[y0:y0+h, x0:x0+w]  #원본 이미지에서 선택 영역만 ROI로 지정 ---⑥
            cv2.imshow('cropped', roi)   #ROI 지정 영역을 새 창으로 표시
            cv2.moveWindow('cropped', 0, 0) #새 창을 화면 좌측 상단으로 이동
            cv2.imwrite('./cropped.jpg', roi) #ROI 영역만 파일로 저장 ---⑦
            print("cropped.")
        else:
            cv2.imshow('img', img)   #드래그 방향이 잘못된 경우 사각형 그림이 없는 원본 이미지 출력
            print("좌측 상단에서 우측 하단으로 영역을 드래그하세요.")
    
img = cv2.imread('../img/sunset.jpg')
cv2.imshow('img', img)
cv2.setMouseCallback('img', onMouse) #마우스 이벤트 등록 ---⑧
cv2.waitKey()
cv2.destroyAllWindows()

x:323, y:158, w:51, h:44
cropped.


 - ROI 지정을 위한 함수(별도의 이벤트 처리를 위한 코드 없어도 됨)
 - ret = cv2.selectROI([win_name,] img[, showCrossHair = True, fromCenter = False])
   * win_name: ROI 선택을 진행할 창의 이름, str
   * img: ROI 선택을 진행할 이미지, Numpy ndarray
   * showCrossHair: 선택 영역 중심에 십자 모양 표시 여부
   * fromCenter: 마우스 시작 지점을 영역의 중심으로 지정
   * ret: 선택한 영역 좌표와 크기(x, y, w, h), 선택을 모두 취소한 경우 0

In [11]:
#practice. selectROI로 관심영역 지정
#영역 드래그 -> esc 누름 -> 선택영역 새 창으로 팝업 & cropped2.jpg로 저장 -> 아무 키 누르면 종료
import cv2
import numpy as np

img = cv2.imread('../img/sunset.jpg')

x, y, w, h = cv2.selectROI('img', img)
if w and h:
    roi = img[y:y+h, x:x+w]
    cv2.imshow('cropped', roi)  #ROI 지정 영역을 새 창으로 표시
    cv2.moveWindow('cropped', 0, 0)   #새 창을 화면 좌측 상단으로 이동
    cv2.imwrite('./cropped2.jpg', roi) #ROI 영역만 파일로 저장
    
cv2.imshow('img', img)
cv2.waitKey(0)
cv2.destroyAllWindows()